# Gerardo de Miguel González

## M1964 Assignment: Building a weather generator

### R implementation

### Description

Weather generators are computational models which generate (daily) synthetic series of random values simulating the distribution of a particular meteorological variable. Precipitation is the most popular example due to its mixed character: discrete (ocurrence: dry or wet) and continuous (rain amount for wet days). The discrete part is typically characterized by the frequency of wet/dry days (say, e.g. 0.53/0.47 in Santander), whereas the continuous is typically characterized by an exponential distribution defined by a rate parameter.

The main control structures (conditionals and loops) will be used and applied to build a simple weather generator and create artificial precipitation amount series. Some functions which simulate (pseudo)random numbers from different families of distributions (uniform and exponential) will also be used. This is a key task for several computational statistics problems.

### How does a precipitation time series look like?

In order to work with real data, we will use two files with daily information for two weather stations: Santander CMT (RR_STAID003922.txt) and Santander/Parayas (RR_STAID003923.txt). The former is located in the Santander regional center (close to el Faro de Cabo Mayor) and the latter is located in Santander airport. 

#### ::GMG::Task 1

I will open Santander CMT dataset (RR_STAID003922.txt) and see the structure. it has some lines with information (to be skipped) and then a header followed by number in columns (STAID: the code of the station; SOUI: the international code of the station; DATA: the day; RR: precipitation value; Q_RR: a quality control flat, 0: OK; 9: missing data.

::GMG::First 20 line metadata exceprt ...

> EUROPEAN CLIMATE ASSESSMENT & DATASET (ECA&D), file created on: 10-11-2017
>
> THESE DATA CAN BE USED FREELY FOR NON-COMMERCIAL RESEARCH PROVIDED THAT THE FOLLOWING SOURCE IS ACKNOWLEDGED: 
>
> Klein Tank, A.M.G. and Coauthors, 2002. Daily dataset of 20th-century surface
air temperature and precipitation series for the European Climate Assessment.
Int. J. of Climatol., 22, 1441-1453.
>
> Data and metadata available at http://www.ecad.eu
>
> FILE FORMAT (MISSING VALUE CODE = -9999):
>
> 01-06 STAID: Station identifier
> 08-13 SOUID: Source identifier
> 15-22 DATE : Date YYYYMMDD
> 24-28 RR   : Precipitation amount in 0.1 mm
> 30-34 Q_RR : quality code for RR (0='valid'; 1='suspect'; 9='missing')
>
> This is the blended series of station SANTANDER CMT, SPAIN (STAID: 3922)
>
> Blended and updated with sources:19972 19981 908023 
>
> See files sources.txt and stations.txt for more info.

#### ::GMG::Task 2

I will load the Santander CMT dataset and plot the first year available of precipitation data (RR column, year 1924 it seems ...). As the precipitation data is in 0.1 mm units so the actual mm (l/m²) is the data value divided by 10. I will also check that the dataset has no suspect or missing values.

In [ ]:
#::GMG::Loading data for Santander CMT
#       Skip first 20 line dataset description and include header with column names:
#
#       01-06 STAID: Station identifier
#       08-13 SOUID: Source identifier
#       15-22 DATE : Date YYYYMMDD
#       24-28 RR   : Precipitation amount in 0.1 mm
#       30-34 Q_RR : quality code for RR (0='valid'; 1='suspect'; 9='missing')
#
#       FILE FORMAT (MISSING VALUE CODE = -9999)
#
precip_s<-read.csv("RR_STAID003922.txt",skip = 20, header = T)

In [19]:
#::GMG::Print the basics of tha dataset
print(paste(class(precip_s),'with',ncol(precip_s),'columns and',nrow(precip_s),'rows.'))
print('Description:')
str(precip_s)
print('First dates ...')
head(precip_s)
print ('... last dates.')
tail(precip_s)

[1] "data.frame with 5 columns and 34273 rows."
[1] "Description:"
'data.frame':	34273 obs. of  5 variables:
 $ STAID: int  3922 3922 3922 3922 3922 3922 3922 3922 3922 3922 ...
 $ SOUID: int  19972 19972 19972 19972 19972 19972 19972 19972 19972 19972 ...
 $ DATE : int  19240101 19240102 19240103 19240104 19240105 19240106 19240107 19240108 19240109 19240110 ...
 $ RR   : int  1 25 7 96 0 0 4 15 65 13 ...
 $ Q_RR : int  0 0 0 0 0 0 0 0 0 0 ...
[1] "First dates ..."


STAID,SOUID,DATE,RR,Q_RR
3922,19972,19240101,1,0
3922,19972,19240102,25,0
3922,19972,19240103,7,0
3922,19972,19240104,96,0
3922,19972,19240105,0,0
3922,19972,19240106,0,0


[1] "... last dates."


,STAID,SOUID,DATE,RR,Q_RR
34268,3922,19981,20171026,0,0
34269,3922,19981,20171027,0,0
34270,3922,19981,20171028,0,0
34271,3922,19981,20171029,21,0
34272,3922,19981,20171030,17,0
34273,3922,19981,20171031,0,0


In [4]:
#::GMG::Basic question about dataset QoS (Q_RR)
#       Q: How many 'suspect' and 'missing' values (RR) are there?
#print(paste('There are',length(which(precip_s$Q_RR == 1)),'suspect values.'))
#print(paste('There are',length(which(precip_s$Q_RR == 9)),'missing values.'))
print(paste('There are',sum(precip_s$Q_RR == 1),'suspect values.'))
print(paste('And there are',sum(precip_s$Q_RR == 9),'missing values.'))
print(paste('So There are',sum(precip_s$Q_RR == 0),'of',length(precip_s$Q_RR),'valid values left.'))
#print(table(precip_s$Q_RR))
#
#::GMG::See
#       https://stackoverflow.com/questions/1923273/counting-the-number-of-elements-with-the-values-of-x-in-a-vector
#       https://www.theanalysisfactor.com/r-tutorial-count/
#
#::GMG::A Question about precipitation amount i.e. 0.1 mm unit in RR column ....
#
#  See  https://earthscience.stackexchange.com/questions/3132/intuitive-explanation-of-precipitation-amount/3175#3175
#       https://earthscience.stackexchange.com/questions/14587/what-does-a-mm-of-rain-mean
#       http://www.bom.gov.au/climate/data-services/content/faqs-elements.html
print(paste('precipitation (mm ~ l/m²) max=',max(precip_s$RR)/10,'min=',min(precip_s$RR)/10))

[1] "There are 0 suspect values."
[1] "And there are 0 missing values."
[1] "So There are 34273 of 34273 valid values left."
[1] "precipitation (mm ~ l/m2) max= 126.2 min= 0"


In [15]:
#::GMG::Ploting data for Santander CMT
#       NOTE:options() needed to fix the plot size in some jupyter notebooks environments
#            (this tested in Microsoft Azure Notebooks)
#       SEE: Blog Revolutions: Resizing plots in the R kernel for Jupyter notebooks
#       https://blog.revolutionanalytics.com/2015/09/resizing-plots-in-the-r-kernel-for-jupyter-notebooks.html
options(repr.plot.width=8, repr.plot.height=6)
plot(precip_s$RR[1:365]/10, 
     type="l",
     xlim = c(1,365), # first year
     ylim = c(0,130),
     ylab = 'precip (mm ~ l/m²)', 
     xlab = 'Day (Year 1924)',
     main = 'Santander CMT')   

#### ::GMG::Task 3

I will load the dataset for Parayas and plot the first year available. The time span of this series is not the same than in the first one. It starts in september 1953 and then spans until 2017. In addition, this series also contains missing data characterized by the number -9999 and Q_RR value of 9. Then again, the units of the data is "10 * mm". I have to divide by 10 in order to get "mm" (liters per square meter).

::GMG::Here it is the 20 line metada

> EUROPEAN CLIMATE ASSESSMENT & DATASET (ECA&D), file created on: 10-11-2017
>
> THESE DATA CAN BE USED FREELY FOR NON-COMMERCIAL RESEARCH PROVIDED THAT THE FOLLOWING SOURCE IS ACKNOWLEDGED: 
>
> Klein Tank, A.M.G. and Coauthors, 2002. Daily dataset of 20th-century surface air temperature and precipitation series for the European Climate Assessment.
>
> Int. J. of Climatol., 22, 1441-1453.
>
> Data and metadata available at http://www.ecad.eu
>
> FILE FORMAT (MISSING VALUE CODE = -9999):
>
> 01-06 STAID: Station identifier
> 08-13 SOUID: Source identifier
> 15-22 DATE : Date YYYYMMDD
> 24-28 RR   : Precipitation amount in 0.1 mm
> 30-34 Q_RR : quality code for RR (0='valid'; 1='suspect'; 9='missing')
>
> This is the blended series of station SANTANDER/PARAYAS, SPAIN (STAID: 3923)
>
> Blended and updated with sources:19990 
>
> See files sources.txt and stations.txt for more info.

Same structure of columns. Let's get this task done.

In [20]:
#::GMG:: Load the series for Parayas and remove missing data.
#       Skip first 20 line dataset description and include header with column names:
#
#       01-06 STAID: Station identifier
#       08-13 SOUID: Source identifier
#       15-22 DATE : Date YYYYMMDD
#       24-28 RR   : Precipitation amount in 0.1 mm
#       30-34 Q_RR : quality code for RR (0='valid'; 1='suspect'; 9='missing')
#
#       FILE FORMAT (MISSING VALUE CODE = -9999)
#
precip_p <- read.csv("RR_STAID003923.txt",skip = 20, header = T)

In [30]:
#::GMG::Print the basics of tha dataset as before ....
print(paste(class(precip_p),'with',ncol(precip_p),'columns and',nrow(precip_p),'rows.'))
print('Description:')
str(precip_p)
print('First dates ...')
head(precip_p)
print ('... last dates.')
tail(precip_p)

[1] "data.frame with 5 columns and 23437 rows."
[1] "Description:"
'data.frame':	23437 obs. of  5 variables:
 $ STAID: int  3923 3923 3923 3923 3923 3923 3923 3923 3923 3923 ...
 $ SOUID: int  19990 19990 19990 19990 19990 19990 19990 19990 19990 19990 ...
 $ DATE : int  19530901 19530902 19530903 19530904 19530905 19530906 19530907 19530908 19530909 19530910 ...
 $ RR   : int  -9999 -9999 -9999 -9999 -9999 -9999 -9999 -9999 -9999 -9999 ...
 $ Q_RR : int  9 9 9 9 9 9 9 9 9 9 ...
[1] "First dates ..."


STAID,SOUID,DATE,RR,Q_RR
3923,19990,19530901,-9999,9
3923,19990,19530902,-9999,9
3923,19990,19530903,-9999,9
3923,19990,19530904,-9999,9
3923,19990,19530905,-9999,9
3923,19990,19530906,-9999,9


[1] "... last dates."


,STAID,SOUID,DATE,RR,Q_RR
23432,3923,19990,20171026,0,0
23433,3923,19990,20171027,1,0
23434,3923,19990,20171028,0,0
23435,3923,19990,20171029,86,0
23436,3923,19990,20171030,34,0
23437,3923,19990,20171031,0,0


In [28]:
#::GMG::Basic question about dataset QoS (Q_RR), again!
#       Q1: How many 'suspect' and 'missing' values (RR) are there?
print(paste('There are',sum(precip_p$Q_RR == 1),'suspect values.'))
print(paste('And there are',sum(precip_p$Q_RR == 9),'missing values.'))
print(paste('So There are',sum(precip_p$Q_RR == 0),'of',length(precip_p$Q_RR),'valid values left.'))
#       Q2: What is max and min precip. save for the'suspect' and 'missing' values?
print(
    paste(
        'precipitation (mm ~ l/m²) max=',max(precip_p$RR)/10,
        'min=',min(precip_p$RR & 
                   precip_p$Q_RR != 1 & 
                   precip_p$Q_RR != 9)/10
    )
)

[1] "There are 0 suspect values."
[1] "And there are 832 missing values."
[1] "So There are 22605 of 23437 valid values left."
[1] "precipitation (mm ~ l/m²) max= 134.4 min= 0"


In [43]:
#::GMG::Ploting data for Santander Parayas
#       NOTE:options() needed to fix the plot size in some jupyter notebooks environments
#            (this tested in Microsoft Azure Notebooks)
#       SEE: Blog Revolutions: Resizing plots in the R kernel for Jupyter notebooks
#       https://blog.revolutionanalytics.com/2015/09/resizing-plots-in-the-r-kernel-for-jupyter-notebooks.html
options(repr.plot.width=8, repr.plot.height=6)
plot(precip_p$RR[123:487]/10, 
     type="l",
     xlim = c(1,365), # year 1954
     ylim = c(0,100),
     ylab = 'precip (mm ~ l/m²)', 
     xlab = 'Day (Year 1954)',
     main = 'Santander Parayas')  

::GMG::**NOTE**: See *the glitch* at the end of the year! There is a missing value ...

In [44]:
precip_p_y_1954<-precip_p$RR[123:487]
str(precip_p_y_1954)
tail(precip_p_y_1954,n = 30)
print(precip_p[which(precip_p_y_1954 == -9999)+122,])

 int [1:365] 0 102 0 128 76 86 214 7 10 165 ...


[1]     0     0     0     0     0    50     8     0   108     0     7 -9999
[13]     0     0     0     0     0     0     0     1    14     8     9     0
[25]     0     0     0     0     0     0

    STAID SOUID     DATE    RR Q_RR
469  3923 19990 19541213 -9999    9


#### Task 4

Both series should be very similar. Compare them using e.g. a scatter plot (points displayed along the diagonal if equal).

 - [QuickR Scatter plots](https://www.statmethods.net/graphs/scatterplot.html)
 
 ::GMG::How to deal with dates in R (i.e. slicing the dataframes by DATE field ...)?
 
 http://www.r-tutor.com/r-introduction/data-frame/data-frame-row-slice
 http://uc-r.github.io/dates/
 https://stackoverflow.com/questions/17518564/convert-yyyymmdd-string-to-date-class-in-r
 https://www.google.com/search?q=how+to+deal+with+dates+as+string+format+yyyymmdd+in+datasets+in+R&ie=utf-8&oe=utf-8&client=firefox-b-ab
 
 https://stackoverflow.com/questions/27607804/yyyymmdd-format